<a href="https://colab.research.google.com/github/nikhilScripts/Machine-learning/blob/main/mail_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## project

Importing the Dependencies

In [30]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

Logistic Regression from scratch

In [31]:
import numpy as np

class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        if hasattr(X, "toarray"):  # handle sparse matrices
            X = X.toarray()

        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_proba(self, X):
        if hasattr(X, "toarray"):
            X = X.toarray()
        linear_model = np.dot(X, self.weights) + self.bias
        return self.sigmoid(linear_model)

    def predict(self, X):
        proba = self.predict_proba(X)
        return [1 if i >= 0.5 else 0 for i in proba]


Accuracy score

In [32]:
def accuracy_score(y_true, y_pred):
    return sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred)) / len(y_true)

In [33]:
import numpy as np

def custom_train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    test_count = int(len(X) * test_size)
    test_indices = indices[:test_count]
    train_indices = indices[test_count:]

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]


Data Collection & Pre-Processing

In [34]:
# dataset
# https://drive.google.com/file/d/1uzbhec5TW_OjFr4UUZkoMm0rpyvYdhZw/view

In [35]:
# loading the data from csv file to a pandas Dataframe
raw_mail_data = pd.read_csv('/content/mail_data.csv')

In [36]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [37]:
# replace the null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [38]:
# printing the first 5 rows of the dataframe
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [39]:
# checking the number of rows and columns in the dataframe
mail_data.shape

(5572, 2)

Label Encoding

In [40]:
# label spam mail as 0;  ham mail as 1;

mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

spam  -  0

ham  -  1

In [41]:
# separating the data as texts and label

X = mail_data['Message']

Y = mail_data['Category']

In [42]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [43]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Splitting the data into training data & test data

In [44]:
X_train, X_test, Y_train, Y_test = custom_train_test_split(X, Y, test_size=0.2, random_state=3)

In [45]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4458,)
(1114,)


Feature Extraction

In [46]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True) # Changed 'True' to True

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [47]:
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [48]:
print(X_train)

1136                      K do I need a login or anything
3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
                              ...                        
789     5 Free Top Polyphonic Tones call 087018728737,...
968     What do u want when i come back?.a beautiful n...
1667    Guess who spent all last night phasing in and ...
3321    Eh sorry leh... I din c ur msg. Not sad alread...
1688    Free Top ringtone -sub to weekly ringtone-get ...
Name: Message, Length: 4458, dtype: object


In [49]:
print(X_train_features)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34777 stored elements and shape (4458, 7431)>
  Coords	Values
  (0, 4557)	0.5258548482573856
  (0, 4033)	0.8505743227750305
  (1, 2329)	0.3878415964861806
  (1, 3811)	0.3478062074993161
  (1, 2224)	0.4131052229068702
  (1, 4456)	0.41686749795649014
  (1, 5413)	0.6198187654585444
  (2, 3811)	0.17420340490417738
  (2, 3046)	0.25037270138182394
  (2, 1991)	0.3303687980535695
  (2, 2956)	0.3303687980535695
  (2, 2758)	0.322639876712425
  (2, 1839)	0.27849087304812625
  (2, 918)	0.22871793310014313
  (2, 2746)	0.3398282332746072
  (2, 2957)	0.3398282332746072
  (2, 3325)	0.31610516682940626
  (2, 3185)	0.29694474756683137
  (2, 4080)	0.1888092515351492
  (3, 6601)	0.6056780583017753
  (3, 2404)	0.452878732195982
  (3, 3156)	0.41072685419977784
  (3, 407)	0.5092724171359624
  (4, 7414)	0.8099979785760476
  (4, 2870)	0.5864326685159318
  :	:
  (4455, 2855)	0.4721050308606525
  (4455, 2246)	0.4721050308606525
  (4456, 4456)	0.249202

Training the Model

Logistic Regression

In [50]:
model = LogisticRegressionScratch()

In [51]:
# training the Logistic Regression model with the training data
model.fit(X_train_features, Y_train)

Evaluating the trained model

In [52]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [53]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.867205024674742


In [54]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [55]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.8608617594254937


Building a Predictive System

In [56]:
input_mail = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

# making prediction

prediction = model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[1]
Ham mail
